# fix the problem with excelfile changes

Background: DISARM's master dataset is in an excelfile. Changes in this dont' show up in github, so it's difficult to tell what's changed between versions.  Code below checks for those differences - use this repeatedly until versions align. 

In [5]:
import pandas as pd
import sqlite3 as sql
from generate_DISARM_pages import Disarm
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import CountVectorizer
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', -1)

newfile = '../DISARM_MASTER_DATA/DISARM_DATA_MASTER.xlsx'
oldfile = '../../DISARMframeworks_2022_08_10/DISARM_MASTER_DATA/DISARM_DATA_MASTER.xlsx'

# Load dfs from file
newdfs = {}
newxlsx = pd.ExcelFile(newfile)
for sheetname in newxlsx.sheet_names:
    newdfs[sheetname] = newxlsx.parse(sheetname)
    newdfs[sheetname].fillna('', inplace=True)

olddfs = {}
oldxlsx = pd.ExcelFile(oldfile)
for sheetname in oldxlsx.sheet_names:
    olddfs[sheetname] = oldxlsx.parse(sheetname)
    olddfs[sheetname].fillna('', inplace=True)

addedtables = newdfs.keys() - olddfs.keys()
losttables = olddfs.keys() - newdfs.keys()
if len(addedtables) + len(losttables) > 0:
    print('Table changes: new tables are {}, lost tables are {}'.format(addedtables, losttables))

def investigate_table(table):
    print('\n\nTable {} is changed'.format(table))
    # Column headings
    coldiffs = set(newdfs[table].columns).symmetric_difference(set(olddfs[table].columns))
    if len(coldiffs) > 0:
        print('column differences: {}'.format(coldiffs))
    # length
    if len(newdfs[table]) != len(olddfs[table]):
        print('length differences: new {} old {}'.format(len(newdfs[table]), len(olddfs[table])))

    # column by column
    for column in newdfs[table].columns:
        coldiffs = newdfs[table][column] != olddfs[table][column]
        if len(newdfs[table][coldiffs]) > 0:
            print('Differences in column {}'.format(column))
    return

for table in newdfs.keys():
    if newdfs[table].equals(olddfs[table]) == False:
        investigate_table(table)
    else:
        print('table {} is the same'.format(table))

<ipython-input-5-7e799212f01e>:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


table README is the same
table examples is the same
table incidents is the same
table incidenttechniques is the same
table narratives is the same
table externalgroups is the same
table externalgroupcounters is the same
table suggested_externalgroups is the same
table removed_externalgroups is the same
table forums is the same
table tools is the same
table removed_tools is the same


In [3]:
# Look at individual table differences
table = 'countermeasures'
column = 'summary'
coldiffs = newdfs[table][column] != olddfs[table][column]
diffcols = pd.DataFrame()
diffcols['disarm_id'] = newdfs[table][coldiffs]['disarm_id']
diffcols['new'] = newdfs[table][coldiffs][column]
diffcols['old'] = olddfs[table][coldiffs][column]
diffcols[diffcols['old'] != '']

,disarm_id,new,old
